In [1]:
import requests
from bs4 import BeautifulSoup

session = requests.Session()

In [2]:
response_queryTime = session.get("https://www.railway.gov.tw/tra-tip-web/tip/tip001/tip119/queryTime")
response_nonPicture = session.get("https://www.railway.gov.tw/tra-tip-web/tip/player/nonPicture")


In [3]:
soup = BeautifulSoup(response_queryTime.content, 'html.parser')
csrf_input = soup.find('form').find('input', {'name': '_csrf'})
csrf_value = csrf_input.get('value')
csrf_value


'08b5f27c-fae6-4d9b-b199-22b690785b98'

In [4]:
from pydub import AudioSegment
import secrets
import io


    
random_value = secrets.randbits(32)
change_code_url = f"https://www.railway.gov.tw/tra-tip-web/tip/player/changeVoiceVerifyCode?pageRandom={random_value}"
response = session.get(change_code_url)

# 下載音頻
random_value = secrets.randbits(32)
audio_url = f"https://www.railway.gov.tw/tra-tip-web/tip/player/audio?pageRandom={random_value}"
response = session.get(audio_url)

# 方法1：直接從內存處理
audio = AudioSegment.from_mp3(io.BytesIO(response.content))

start_time = 15500  
end_time = 32500    
audio_slice = audio[start_time:end_time]


output_wav_path = f"captcha_audio_slice_{random_value}.wav"

audio_slice.export(output_wav_path, format='wav')
print(f"已轉換: {output_wav_path}")

已轉換: captcha_audio_slice_1917236868.wav


In [5]:
import azure.cognitiveservices.speech as speechsdk
import time

            
def speech_recognize_continuous_from_file(file_path):
    
    result_text = []

            
    def result_callback(event_type: str, evt: speechsdk.SpeechRecognitionEventArgs):
        """callback to display a translation result"""
        nonlocal result_text
        if event_type == 'RECOGNIZING':
            pass
        elif event_type == 'RECOGNIZED':
            result_text.append(evt.result.text)
            
    """performs continuous speech recognition with input from an audio file"""
    # <SpeechContinuousRecognitionWithFile>
    speech_config = speechsdk.SpeechConfig(subscription='f9d5f7f373ab494b8a707016cdf994d7', region='eastasia')
    speech_config.speech_recognition_language="zh-TW"
    audio_config = speechsdk.audio.AudioConfig(filename=file_path)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    done = False

    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        nonlocal done
        done = True

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: result_callback("RECOGNIZING", evt))
    speech_recognizer.recognized.connect(lambda evt: result_callback("RECOGNIZED", evt))
    # speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
    # speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
    speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # Stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()
    
    return result_text
    # </SpeechContinuousRecognitionWithFile>

In [6]:
result_text = speech_recognize_continuous_from_file(output_wav_path)
result_text

CANCELED SpeechRecognitionCanceledEventArgs(session_id=e969afceed754aa6b6110d9d0a195227, result=SpeechRecognitionResult(result_id=c9f28f2f8914479880a7d267071235ea, text="", reason=ResultReason.Canceled))
CLOSING on SpeechRecognitionCanceledEventArgs(session_id=e969afceed754aa6b6110d9d0a195227, result=SpeechRecognitionResult(result_id=c9f28f2f8914479880a7d267071235ea, text="", reason=ResultReason.Canceled))
CLOSING on SessionEventArgs(session_id=e969afceed754aa6b6110d9d0a195227)


['RT。', 'P。', 'MHX。', '現在播放驗證碼第。']

In [7]:
import string
def remove_punctuation(text):
    # 英文標點
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 中文標點
    chinese_punc = '，。！？【】（）《》""''；：、'
    text = text.translate(str.maketrans('', '', chinese_punc))
    
    # 去除多餘空格
    text = ''.join(text.split())
    
    return text

result_text = ' '.join(result_text)
result_text = remove_punctuation(result_text)
result_text

'RTPMHX現在播放驗證碼第'

In [8]:
if '現在' in result_text:
    result_text = result_text.split('現在')[0]
else:
    raise Exception('無法從音頻中識別出現在')

if len(result_text) != 6:
    raise Exception('驗證碼長度不正確')

result_text


'RTPMHX'

In [9]:
num_map = {
    '零': '0', '一': '1', '二': '2', '三': '3', '四': '4',
    '五': '5', '六': '6', '七': '7', '八': '8', '九': '9'
}

for word, num in num_map.items():
    result_text = result_text.replace(word, num)

result_text = result_text.lower()
result_text


'rtpmhx'

In [10]:
search_url = "https://www.railway.gov.tw/tra-tip-web/tip/tip001/tip119/search"

form_data = {
    '_csrf': csrf_value,  # 從之前的請求中獲取
    'rideDate': '2024/11/17',
    'startStation': '0930-七堵',
    'endStation': '4310-岡山',
    'startOrEndTime': 'true',
    'startTime': '15:00',
    'endTime': '23:00',
    'g-recaptcha-response': None, # 這個需要從實際的 reCAPTCHA 響應中獲取
    'hiddenRecaptcha': '',
    'verifyType': 'voice',
    'verifyCode': result_text
}


In [11]:
response = session.post(
    search_url,
    data=form_data,
)
# 檢查響應
if response.status_code == 200:
    print("請求成功")
    # 解析響應內容
    soup = BeautifulSoup(response.content, 'html.parser')
    # 處理響應...
else:
    print(f"請求失敗: {response.status_code}")

請求成功


In [12]:
soup = BeautifulSoup(response.content, 'html.parser')

In [13]:
table = soup.find('table', {'class': 'itinerary-controls'})

if table:
    print("找到表格：")
    trips = table.find_all(class_='trip-column')
    if trips:
        print("車次資訊：")
        for trip in trips:
            # 取得基本資訊（直接從 td 元素）
            tds = trip.find_all('td')
            
            # 取得車種車次（從 a 元素）
            train_info = trip.find('a', {'class': 'links'})
            
            # 取得票價
            price = trip.find('span').text.strip() if trip.find('span') else "N/A"
            
            ticket_status_class_names = tds[6].find('div', {'class': 'icon-fa'}).get('class')
            if  'times' in ticket_status_class_names and 'red' in ticket_status_class_names:
                ticket_status = '沒票'
            elif 'exclamation-triangle' in ticket_status_class_names and 'color-red' in ticket_status_class_names:
                ticket_status = '有票 < 30'
            elif 'green' in ticket_status_class_names and 'check-circle' in ticket_status_class_names:
                ticket_status = '有票 > 30'
            else:
                ticket_status = '未知'
            # 整理資訊
            train_data = {
                "出發時間": tds[0].text.strip(),
                "抵達時間": tds[1].text.strip(),
                "旅程時間": tds[2].text.strip(),
                "車種車次": train_info.text.strip() if train_info else "N/A",
                "經由": tds[4].text.strip(),
                "餘票狀態": ticket_status,
                "全票票價": tds[7].find('span').text.strip(),
                "優待票價": tds[8].find('span').text.strip()
            }
            print(train_data)
    else:
        print("找不到車次資訊")
else:
    print("找不到指定的表格")
    
    # 輸出整個 HTML 以便檢查
    print("\n頁面內容：")
    print(soup.prettify()[:500])  # 只顯示前500字符

找到表格：
車次資訊：
{'出發時間': '15:30', '抵達時間': '20:38', '旅程時間': '05時08分', '車種車次': '自強175', '經由': '山', '餘票狀態': '沒票', '全票票價': '$ 845', '優待票價': '$ 423'}
{'出發時間': '15:41', '抵達時間': '22:30', '旅程時間': '06時49分', '車種車次': '莒光521', '經由': '海', '餘票狀態': '沒票', '全票票價': '$ 652', '優待票價': '$ 326'}
{'出發時間': '17:38', '抵達時間': '22:41', '旅程時間': '05時03分', '車種車次': '自強(3000)145', '經由': '海', '餘票狀態': '沒票', '全票票價': '$ 845', '優待票價': '$ 423'}
